In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device,enable=True)
from tensorflow.keras.layers import BatchNormalization
# from keras.layers.normalization import layer_normalization
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

2022-01-04 11:38:58.210133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 11:38:58.273464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 11:38:58.273691: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
N_train = 1000
X = np.expand_dims(np.zeros((1200,20),dtype=float), axis=2)
Y = np.expand_dims(np.zeros((1200),dtype=float), axis=1)
# Y = np.zeros(1200,dtype=float)
X_train = X[:1000, :]
Y_train = Y[:1000, :]
X_test = X[1000:, :]
Y_test = Y[1000:, :]


In [4]:
from numpy.random import choice
one_indexes = choice(a=N_train, size=int(N_train / 2), replace=False)
X_train[one_indexes, 0, :] = 1  # very long term memory.
Y_train[one_indexes, :] = 1  # very long term memory.

one_indexes = choice(a=200, size=int(200 / 2), replace=False)
X_test[one_indexes, 0, :] = 1  # very long term memory.
Y_test[one_indexes] = 1  # very long term memory.


In [5]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)


In [6]:
x_train, y_train = prepare_sequences(X_train, Y_train, 10)
x_test, y_test = prepare_sequences(X_test, Y_test, 10)



In [7]:
max_len= 10
batch_size = 64
print('Building STATELESS model...')
model = Sequential()
model.add(LSTM(10, input_shape=(max_len, 1), return_sequences=False, stateful=False))
# model.add(LSTM(10, batch_input_shape=(batch_size, max_len, 1), return_sequences=False, stateful=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# train_size = batch_size * x_train.shape

Building STATELESS model...


2021-12-28 18:52:08.091091: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-28 18:52:08.091926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-28 18:52:08.092105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-28 18:52:08.092224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [8]:
# x_train1 = x_train[:170*batch_size,:,:]
# y_train1 = y_train[:170*batch_size,:]
# x_test1 = x_test[:33*batch_size,:,:]
# y_test1 = y_test[:33*batch_size,:]
#
# model.fit(x_train1, y_train1, batch_size=batch_size, epochs=15,
#           validation_data=(x_test1, y_test1), shuffle=False)
model.fit(x_train, y_train, batch_size=batch_size, epochs=15,
          validation_data=(x_test, y_test), shuffle=False)

Epoch 1/15


2021-12-28 18:52:09.788643: E tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded runtime CuDNN library: 8.0.5 but source was compiled with: 8.1.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2021-12-28 18:52:09.789308: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1555 : UNKNOWN: Fail to find the dnn implementation.


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_train_function_3062]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)